In [47]:
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import wave
import numpy
from music21 import converter, instrument, note, chord
import glob
import os
import Paths
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense, LSTM, MaxPooling2D, Dropout, Activation

training = os.listdir(Paths.IN_DIRECTORY)
array = []
for sample_name in training:
    if 'wav' in sample_name:
        file_path = os.path.join(Paths.IN_DIRECTORY, sample_name)
        song = AudioSegment.from_wav(file_path)
        samples = song.get_array_of_samples()
        array.append(samples[:10])

flatlist=[]
for sublist in array:
    for element in sublist:
        flatlist.append(element)

# sequence_length = len(song)
sequence_length = 100

# get all pitch names
pitchnames = sorted(set(item for item in flatlist))

n_vocab = len(set(flatlist))

# create a dictionary to map pitches to integers
note_to_int = dict((flatlist, number) for number, flatlist in enumerate(pitchnames))

network_input = []
network_output = []

# create input sequences and the corresponding outputs
for i in range(0, len(flatlist) - sequence_length, 1):
    sequence_in = flatlist[i:i + sequence_length]
    sequence_out = flatlist[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

model = Sequential()
model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
    filepath, monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)
callbacks_list = [checkpoint]
model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

Epoch 1/200
27/27 [==============================] - 27s 855ms/step - loss: 7.3394
Epoch 2/200
27/27 [==============================] - 24s 872ms/step - loss: 7.3292
Epoch 3/200
27/27 [==============================] - 24s 904ms/step - loss: 7.3277
Epoch 4/200
27/27 [==============================] - 25s 925ms/step - loss: 7.3219
Epoch 5/200
27/27 [==============================] - 25s 942ms/step - loss: 7.2764
Epoch 6/200
27/27 [==============================] - 27s 985ms/step - loss: 7.2232
Epoch 7/200
27/27 [==============================] - 26s 971ms/step - loss: 7.1547
Epoch 8/200
27/27 [==============================] - 26s 944ms/step - loss: 7.0692
Epoch 9/200
27/27 [==============================] - 26s 957ms/step - loss: 6.9795
Epoch 10/200
27/27 [==============================] - 26s 960ms/step - loss: 6.8696
Epoch 11/200
27/27 [==============================] - 27s 986ms/step - loss: 6.8076
Epoch 12/200
27/27 [==============================] - 26s 978ms/step - loss: 6.6664
E

KeyboardInterrupt: 

In [52]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('weights-improvement-59-2.5657-bigger.hdf5')

ValueError: Cannot assign value to variable ' lstm_33/lstm_cell_33/kernel:0': Shape mismatch.The variable shape (1, 2048), and the assigned value shape (1, 1024) are incompatible.

In [33]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

506


In [57]:
network_input.shape[1]

100

In [58]:
network_input.shape[2]

1

In [54]:
network_output.shape

(1670, 1521)